In [33]:
# program to calculate eigen values or tridagnal matrix using jacobi rotation method
import numpy as np
from scipy import linalg
import time as time
import sys

#Toeplitz for buckling beam supported at both ends

def basicToeplitz_Matrix(ru_min,ru_max,n):
    h = (ru_max - ru_min)/n
    A= np.zeros((n,n), dtype=float)
    a = 2.0/h**2
    b = -1.0/h**2
    A[0,0]=a
    A[0,1]=b
    A[n-1,n-1]=a
    A[n-1,n-2]=b
    for i in range(1,n-1):
        A[i,i]=a
        A[i,i+1] = b
        A[i,i-1]=b
    #print (A)
    return A
    ###################
#def lamda(n)
#    lamda=np.zeros((4,4) , dtype=float) 
#    for i in range (1 ,4):
#        lamda[i,i]=a+2*b*np.cos((i*3.14)/(n+1))
#        print (lamda)
 #       print()
    ##############
    

   # print()

# calculate maximum off diagnal value to cal min rotational nagle 
def maxoffdia(A,n):
    maximum = 0
   # k = 0
   # l = 1
    for i in range(0,n):
        for j in range(i+1,n):
            if(np.abs(A[i][j]) > maximum):
                maximum = np.abs(A[i][j])
                k = i
                l = j
    return maximum,l,k

# jacobi rotation method

def rotate(A,R,k,l,n):
    if (A[k][l] != 0):
        tau = (A[l][l] - A[k][k])/(2*A[k][l])
            # to get smallest abs value of rot angle 
        if (tau >= 0):
            t=1/(tau+np.sqrt(1+tau*tau))
        else:
            t=-1/(-tau+np.sqrt(1+tau*tau))
        c = 1/np.sqrt(1+t**2)
        s = c*t
    else:
        # means a[k][l] is zero
            c = 1.0
            s = 0.0
   
    
    # calculating other elements in the array affected bythe rotation 
    for i in range(0,n):
            if i!=k and i!=l:
                a_ik = A[i][k]
                a_il = A[i][l]
                A[i][k] = c*a_ik - s*a_il
                A[k][i] = A[i][k]
                A[i][l] = c*a_il + s*a_ik
                A[l][i] = A[i][l]
            r_ik = R[i][k]
            r_il = R[i][l]
            R[i][k] = c*r_ik - s*r_il
            R[i][l] = c*r_il +s*r_ik
 # pre calculation of rotations manually defined by k and l 
    a_kk = A[k][k]
    a_ll = A[l][l]
    A[k][k] = c*c*a_kk - 2.0*c*s*A[k][l] + s*s*a_ll
    A[l][l] = s*s*a_kk + 2.0*c*s*A[k][l] + c*c*a_ll
    A[k][l] = 0.0
    A[l][k] = 0.0
   # print()
    #print(A)
##################       
# initialise array 
def jacobi_method(A, R, n):
    for i in range(0,n):
        for j in range(0,n):
            if i==j:
                R[i][j] = 1.0
            else:
                R[i][j] = 0.0
    # max error tolenrance to stop the algorithm 
    tol = 1.0e-10
    # max number of iterations 
    pow_n= 6
    max_iteration = n**pow_n
    iteration = 0
    max_offdiagonal,l,k = maxoffdia(A,n)
    
    start_time_jacobi = time.time()
    
    while abs(max_offdiagonal) > tol and iteration < max_iteration :
        max_offdiagonal,l,k = maxoffdia(A,n)
        # print(max_offdiagonal,l,k)
        rotate(A,R,k,l,n)
        iteration +=1
        
   # print(iteration)
       # print('number of iteration: ' ,iteration , '\n')
    end_time_jacobi=time.time()
    # calculting time for jacobi rotation method 
    exe_time_jacobi=end_time_jacobi-start_time_jacobi
   
   # print(A)
    # eigen array and sorting it 
    eigen=np.zeros((n), dtype=float)
    for i in range(0,n):
        eigen[i]=A[i][i]
    eigen=np.sort(eigen)
    
    print()
    print()
    print("calculated eigen values from jacobi")
    print(eigen)
    
      
    # calculating time for built in linalg function 
    start_time_builtin_linalg_fn=time.time()
    w,v = linalg.eig(A)
    w=np.sort(w)
    end_time_builtin_linalg_fn=time.time()
    exe_time_builtin_linalg_fn= end_time_builtin_linalg_fn-start_time_builtin_linalg_fn
    
    print()
    print()
    print("functional eigen values from numpy")
    print (w)
   # print()
   # print()
    
   # print("execution time jacobi")
   # print(exe_time_jacobi)
   # print("execution time builtin line alg")
    #print(exe_time_builtin_linalg_fn)
    return exe_time_jacobi,exe_time_builtin_linalg_fn,iteration,eigen[0], eigen[1],eigen[2] 
 #   print(max_iteration)
    
    #########
  # rotation matrix   
def Creat_rotate_matrix(n):
    R = np.zeros((n,n), dtype=float)
    return R
####
"""
def test_jacobi():
    A = np.array([[3,2,0,0],[2,7,2,0], [0,2,9,2], [0,0,2,6]], dtype=float)
    print ("unit test")
    print ("A")
    print(A)
    x,y=linalg.eig(A)
    x=np.sort(x)
    print("eigen values from numpy")
    print(x)
    #print()
    R = np.zeros((4,4), dtype=float)
   # R = np.array([[1.,0.,0.,0.],[0.,1.,0.,0.], [0.,0.,1.,0.], [0.,0.,0.,1.]])
    n = 4
    jacobi_method(A, R, n)
    result=np.zeros((n), dtype=float)
    for i in range(0,n):
        result[i]=A[i][i]
    result=np.sort(result)
    
    print("eigen vlues from jacobi's method")
    print(result)
#########
"""

#writing time values to file 
outf = open('out.txt', 'w+')
outf.write("n \t Rho_max\t exe time_jacobi\t exe_time_linealg fn\t no.of transformations\t first three eigen values\n")
outf.close()
for i in range(50 , 100 , 50):
    n =i
    rho_max=10
    R = Creat_rotate_matrix(n)
    A = basicToeplitz_Matrix(0,rho_max,n) 
                #print (n)
    print()
    print()
                #print (R)
                #max_offdiagonal,l,k = maxoff(A,n)
                  #  print(max_offdiagonal,l,k)
    J,F,I, E0, E1,E2=jacobi_method(A, R, n)
    
    
#######################
#test_jacobi()
        
        


    
###################################    
outf=open('out.txt', 'a')
outf.write(str(n))
outf.write("\t")
outf.write(str(rho_max))
outf.write("\t")
outf.write("{:.6f}".format(J))
outf.write("\t")
outf.write("{:.6f}".format(F))
outf.write("\t")
outf.write("{:.0f}".format(I))
outf.write("\t")
outf.write("{:.6f}".format(E0))
outf.write("\t")
outf.write("{:.6f}".format(E1))
outf.write("\t")
outf.write("{:.6f}".format(E2))
outf.write("\n")
    #print(i)
outf.close()






calculated eigen
[9.48335631e-02 3.78974516e-01 8.51345016e-01 1.51015320e+00
 2.35289998e+00 3.37638853e+00 4.57673641e+00 5.94939029e+00
 7.48914321e+00 9.19015438e+00 1.10459713e+01 1.30495541e+01
 1.51933027e+01 1.74690850e+01 1.98682682e+01 2.23817514e+01
 2.50000000e+01 2.77130822e+01 3.05107063e+01 3.33822600e+01
 3.63168505e+01 3.93033458e+01 4.23304173e+01 4.53865820e+01
 4.84602471e+01 5.15397529e+01 5.46134180e+01 5.76695827e+01
 6.06966542e+01 6.36831495e+01 6.66177400e+01 6.94892937e+01
 7.22869178e+01 7.50000000e+01 7.76182486e+01 8.01317318e+01
 8.25309150e+01 8.48066973e+01 8.69504459e+01 8.89540287e+01
 9.08098456e+01 9.25108568e+01 9.40506097e+01 9.54232636e+01
 9.66236115e+01 9.76471000e+01 9.84898468e+01 9.91486550e+01
 9.96210255e+01 9.99051664e+01]


functional eigen
[9.48335631e-02+0.j 3.78974516e-01+0.j 8.51345016e-01+0.j
 1.51015320e+00+0.j 2.35289998e+00+0.j 3.37638853e+00+0.j
 4.57673641e+00+0.j 5.94939029e+00+0.j 7.48914321e+00+0.j
 9.19015438e+00+0.j 1.